In [1]:
%load_ext bigdata

In [2]:
%hive_start

In [3]:
%timeout 300

In [4]:
!hadoop fs -rm -f -r tmp;

In [5]:
## Borra la carpeta si existe
!hdfs dfs -rm -r -f /tmp/output
## Crea la carpeta drivers en el HDFS
!hdfs dfs -mkdir /tmp/output
## Copia los archivos al HDFS
!hdfs dfs -copyFromLocal data.tsv  /tmp/output/data.tsv
## Lista los archivos al HDFS para verificar
## que los archivos fueron copiados correctamente.
!hdfs dfs -ls /tmp/output/*

copyFromLocal: `data.tsv': No such file or directory
ls: `/tmp/output/*': No such file or directory


In [6]:
%%hive
DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;


DROP TABLE IF EXISTS tbl0;
OK
Time taken: 19.748 seconds
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 1.449 seconds
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
Loading data to table default.tbl0
OK
Time taken: 4.38 seconds


In [7]:
%%hive
--
-- para guardar la consulta se debe almacenar en un tabla
DROP TABLE IF EXISTS salida;
CREATE TABLE salida
AS
SELECT 
    year,
    letra,
    count(1) 
FROM (
SELECT
       YEAR(c4) AS year,
       c0 AS letra
FROM
        tbl0  
LATERAL VIEW
        explode(c5) tbl0 AS c0
)tabla
GROUP BY 
    year, letra;

--
-- para guardar la consulta se debe almacenar en un tabla
DROP TABLE IF EXISTS salida;
OK
Time taken: 0.018 seconds
CREATE TABLE salida
AS
SELECT 
    year,
    letra,
    count(1) 
FROM (
SELECT
       YEAR(c4) AS year,
       c0 AS letra
FROM
        tbl0  
LATERAL VIEW
        explode(c5) tbl0 AS c0
)tabla
GROUP BY 
    year, letra;
Query ID = root_20200120031919_a6df0a94-ddd8-446b-ba67-19e88afee6c9
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1579490012394_0001, Tracking URL = http://dc1281897b15:8088/proxy/application_1579490012394_0001/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1579490012394_000

In [8]:
%%hive
DROP TABLE IF EXISTS tbl0;

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 0.779 seconds


In [9]:
%%hive
-- 
INSERT OVERWRITE DIRECTORY 'tmp/000000_0'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT
    *
FROM
    salida;

-- 
INSERT OVERWRITE DIRECTORY 'tmp/000000_0'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT
    *
FROM
    salida;
Query ID = root_20200120032019_ab52830b-48f3-4a42-81eb-c28f3e3a7059
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1579490012394_0002, Tracking URL = http://dc1281897b15:8088/proxy/application_1579490012394_0002/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1579490012394_0002
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-01-20 03:20:35,946 Stage-1 map = 0%,  reduce = 0%
2020-01-20 03:20:44,245 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 2.79 sec
MapReduce Total cumulative CPU time: 2 seconds 790 msec
Ended Job = job_1579490012394_0002
Stage-3 is selected by condition resolver.
Stage-2 is filtered out by condition resolver.
Stage-4 is filtered out by condition resolver.
Moving data to directory hdfs://0.0.0.0:9000/user/root/tmp/000

In [10]:
!hadoop fs -copyToLocal ./tmp/000000_0 output

In [11]:
%hive_quit